In [ ]:
import pandas as pd
import pathlib
import numpy as np
from pprint import pprint
from functools import partial

from {{ cookiecutter: module_name }}.data import datasets, utils, Dataset
from {{ cookiecutter: module_name }}.data.datasets import (build_dataset_dict, fetch_and_unpack, fetch_text_file, read_space_delimited,
                                   load_dataset)
from {{ cookiecutter: module_name }}.data.utils import hash_file, list_dir, head_file, normalize_labels
from {{ cookiecutter: module_name }}.paths import interim_data_path, raw_data_path, processed_data_path

In [ ]:
%load_ext autoreload
%autoreload 2

# Adding and Processing Natural Datasets
## The LVQ-PAK Finnish Phonetic dataset

The Learning Vector Quantization project includes a simple Finnish phonetic dataset
consisting 20-dimensional data and their associated targets. Let's explore this dataset and
add it to our global `datasets.json` so it can be unpacked and processed automatically.

In [ ]:
dataset_name='lvq-pak'

Download the tarfile and build the dataset dictionary for it. If we know the hash of this file, we should include it here. If not, one will be computed from this download and used for comparison on subsequent downloads.

In [ ]:
# Grab the source code package
lvq_pak = build_dataset_dict(url="http://www.cis.hut.fi/research/lvq_pak/lvq_pak-3.1.tar")
lvq_pak

The **name** field can be used to indicate the type of datafile being downloaded. Usually, this is just informational. However, if you specify names `DESCR` or `LICENSE`, the downloaded (text) file will be used as the dataset description and license text, respectively.

Usually you will want to give these unique names, so they don't clash with other downloaded files. (e.g. "LICENSE.txt" is a terrible name to use). We use the **file_name** option for this:

In [ ]:
descr = build_dataset_dict(url='http://www.cis.hut.fi/research/lvq_pak/README', file_name=f'{dataset_name}.readme',
                       name='DESCR')

In [ ]:
# notice the files have been downloaded to the RAW directory
list_dir(raw_data_path)

Next, we combine the complete set of files into a URL list and use this to build our json file entry.

In [ ]:
url_list = [lvq_pak, descr]
url_list

In [ ]:
newds_dict = datasets.add_dataset_by_urllist(dataset_name, url_list)
pprint(newds_dict)

See that a generic `load_function` (`new_dataset`) has been used to process the data. This does nothing more than populates the DESCR and LICENSE fields (if possible), creating an otherwise empty `Dataset` object

In [ ]:
# Now, call the (generic) load function and notice that the LICENSE and DESCR have been set
dset = newds_dict['load_function']()
type(dset)

In [ ]:
print(dset.DESCR)

In [ ]:
license = getattr(dset, 'LICENSE', None)
print(license)

Datasets should *always* have an explicit license. Reading the project documentation, we see a license in one of the textfiles. We can extract and use that

In [ ]:
license_txt = '''
************************************************************************
*                                                                      *
*                              LVQ_PAK                                 *
*                                                                      *
*                                The                                   *
*                                                                      *
*                   Learning  Vector  Quantization                     *
*                                                                      *
*                          Program  Package                            *
*                                                                      *
*                   Version 3.1 (April 7, 1995)                        *
*                                                                      *
*                          Prepared by the                             *
*                    LVQ Programming Team of the                       *
*                 Helsinki University of Technology                    *
*           Laboratory of Computer and Information Science             *
*                Rakentajanaukio 2 C, SF-02150 Espoo                   *
*                              FINLAND                                 *
*                                                                      *
*                      Copyright (c) 1991-1995                         *
*                                                                      *
************************************************************************
*                                                                      *
*  NOTE: This program package is copyrighted in the sense that it      *
*  may be used for scientific purposes. The package as a whole, or     *
*  parts thereof, cannot be included or used in any commercial         *
*  application without written permission granted by its producents.   *
*  No programs contained in this package may be copied for commercial  *
*  distribution.                                                       *
*                                                                      *
*  All comments concerning this program package may be sent to the     *
*  e-mail address 'lvq@nucleus.hut.fi'.                                *
*                                                                      *
************************************************************************
'''

In [ ]:
url_list += [datasets.build_dataset_dict(from_txt=license_txt, file_name=f'{dataset_name}.license', name='LICENSE')]
url_list

Now, reload the dataset from scratch and check that the license is there

In [ ]:
newds_dict = datasets.add_dataset_by_urllist(dataset_name, url_list)
dset = datasets.load_dataset(dataset_name)
print(dset.LICENSE)

## Processing the data
The next step is to write the importer that actually processes the data we will be using for this dataset.

The important things to generate are `data` and `target` entries. A `metadata` is optional, but recommended if you want to save additional information about the dataset.

Usually, this functionality gets bundled up into a function and added to `datasets.py`


In [ ]:
# Unpack the file
untar_dir = fetch_and_unpack(dataset_name)
unpack_dir = untar_dir / 'lvq_pak-3.1'
list_dir(unpack_dir)

In this dataset, the training and test datsets are stored in files named `ex1.dat` and `ex2.dat` respectively

In [ ]:
datafile_train = unpack_dir / 'ex1.dat'
datafile_test = unpack_dir / 'ex2.dat'

datafile_train.exists() and datafile_test.exists()

According to the documentation, the data format is space-delimited, with the class label included as the last column. Let's have a look

In [ ]:
print(head_file(datafile_train))

Indeed, the datafile consists of 1 line containing the dimension of the data, a comment, and then 21 space-delimited columns, the final column being the target class label. 

**Note:** We have to be a little careful importing the data, because '#' is used both as the comment delimiter, and as a class label.

Fortunately, we have a helper function for this. We will get a little cheeky and skip the first 2 lines (hoping there are no other comments). The documentation also says ther are 1962 entries in each of the training and test datasets.

In [ ]:
data, target = read_space_delimited(datafile_train, skiprows=[0,1])
data2, target2 = read_space_delimited(datafile_test, skiprows=[0])

data.shape, target.shape, data2.shape, target2.shape

In [ ]:
target

This seems to work, so let's wrap this functionality up into a processing function.
By convention, the function takes a `dataset_name`, and any other options that may be useful for reading the data, and returns a dictionary that matches the `Dataset` constructor signature.

We will place this function in `localdata.py`, (and add it to `__all__`) to make it visible to our dataset code.

In [ ]:
%%file ../src/data/localdata.py
"""
Custom dataset processing/generation functions should be added to this file
"""

from {{ cookiecutter: module_name }}.data.utils import read_space_delimited, normalize_labels
from {{ cookiecutter: module_name }}.paths import interim_data_path
import numpy as np

__all__ = ['process_lvq_pak']

def process_lvq_pak(dataset_name='lvq-pak', kind='all', numeric_labels=True, metadata=None):
    """
    kind: {'test', 'train', 'all'}, default 'all'
    numeric_labels: boolean (default: True)
        if set, target is a vector of integers, and label_map is created in the metadata
        to reflect the mapping to the string targets
    """
    
    untar_dir = interim_data_path / dataset_name
    unpack_dir = untar_dir / 'lvq_pak-3.1'

    if kind == 'train':
        data, target = read_space_delimited(unpack_dir / 'ex1.dat', skiprows=[0,1])
    elif kind == 'test':
        data, target = read_space_delimited(unpack_dir / 'ex2.dat', skiprows=[0])
    elif kind == 'all':
        data1, target1 = read_space_delimited(unpack_dir / 'ex1.dat', skiprows=[0,1])
        data2, target2 = read_space_delimited(unpack_dir / 'ex2.dat', skiprows=[0])
        data = np.vstack((data1, data2))
        target = np.append(target1, target2)
    else:
        raise Exception(f'Unknown kind: {kind}')

    if numeric_labels:
        if metadata is None:
            metadata = {}
        mapped_target, label_map = normalize_labels(target)
        metadata['label_map'] = label_map
        target = mapped_target

    dset_opts = {
        'dataset_name': dataset_name,
        'data': data,
        'target': target,
        'metadata': metadata
    }
    return dset_opts


Let's make sure this works as expected

In [ ]:
from {{ cookiecutter: module_name }}.data.localdata import process_lvq_pak

for kind in ['train', 'test', 'all']:
    dset_opts = process_lvq_pak(kind=kind)
    dset = Dataset(**dset_opts)
    print(f'{kind}: data={dset.data.shape} target={dset.target.shape}')

This all looks good


In [ ]:
datasets.add_dataset_from_function(dataset_name, process_lvq_pak);

Finally, re-load the dataset and save a copy of it

In [ ]:
lvq = load_dataset(dataset_name)
print(str(lvq))
lvq.dump()

The saved data is stored in the `processed_data_path`. An copy of just the metadata is also stored, so that it may be quickly checked without loading the entire dataset.

In [ ]:
list_dir(processed_data_path)